In [16]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [33]:
# load data

cross = pd.read_csv('20201125_cross.csv', low_memory=False)
cross = cross.drop([0,1])
cross = cross.reset_index()

cross = cross.replace('AA', -1)
cross = cross.replace('AB', 0)
cross = cross.replace('BB', 1)
cross = cross.replace('-', np.nan)

cross.GENERATION = ['F'+str(i) for i in cross.GENERATION]
cross.DAM = ['D'+str(i) for i in cross.DAM]

cross = cross.loc[cross.BW8.isna()==False]
cross.BW8 = cross.BW8.astype(float)
cross.SEX = cross.SEX.astype(float)


cnew = []
for i in cross.columns:
    if '-' in i:
        ni = 'mrk_'+'_'.join(i.split('-'))
    else:
        ni = i
    cnew.append(ni)
    
cross.columns = cnew


cross_markers = [i for i in cross.columns if i.startswith('mrk')]


In [34]:
cross

,index,id,id.1,BW8,BW8_norm_by_gen,GENERATION,SEX,DAM,mrk_1_0,mrk_1_1,...,mrk_29_3,mrk_29_4,mrk_29_5,mrk_29_6,mrk_30_0,mrk_30_1,mrk_30_2,mrk_30_3,mrk_30_4,mrk_30_5
0,2,44902.0,44902,240.0,-0.0111226688106612,F2,1.0,D32201,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0
1,3,51202.0,51202,688.0,0.00178673139483582,F2,1.0,D32101,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0
2,4,120502.0,120502,701.0,0.0021613345257989,F2,0.0,D15101,-1.0,-1.0,...,1.0,1.0,NaN,NaN,NaN,NaN,-1.0,-1.0,-1.0,-1.0
3,5,48402.0,48402,535.0,-0.0026220593003451,F2,0.0,D29201,NaN,NaN,...,-1.0,-1.0,-1.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0
4,6,41402.0,41402,571.0,-0.00158469678383194,F2,0.0,D32001,-1.0,-1.0,...,1.0,1.0,1.0,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3130,3132,351803.0,351803,506.0,-0.00661929165534307,F3,0.0,D138102,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3131,3133,341903.0,341903,898.0,0.00700232797787902,F3,1.0,D142202,NaN,NaN,...,1.0,1.0,1.0,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3132,3134,325803.0,325803,908.0,0.00734981827464489,F3,1.0,D133202,NaN,1.0,...,-1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3133,3135,355903.0,355903,570.0,-0.0043953537560415,F3,1.0,D141002,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0


In [35]:
data = cross
pheno = 'BW8'
md = smf.mixedlm("{pheno} ~ SEX".format(pheno=pheno), data=data, groups='GENERATION')
mdf = md.fit()

In [21]:
mdf.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
=============================================================
Model:              MixedLM  Dependent Variable:  BW8        
No. Observations:   3073     Method:              REML       
No. Groups:         13       Scale:               22246.2055 
Min. group size:    32       Log-Likelihood:      -19759.1085
Max. group size:    875      Converged:           Yes        
Mean group size:    236.4                                    
-------------------------------------------------------------
                Coef.   Std.Err.   z    P>|z|  [0.025  0.975]
-------------------------------------------------------------
Intercept       567.919   27.733 20.478 0.000 513.564 622.275
SEX             141.185    5.429 26.006 0.000 130.545 151.825
GENERATION Var 9634.185   26.875                             
=============================================================

"""

In [20]:
mdf.resid

0      -459.601144
1       -11.601144
2       142.583840
3       -23.416160
4        12.583840
           ...    
3130   -132.547148
3131    118.267868
3132    128.267868
3133   -209.732132
3134    -40.547148
Length: 3073, dtype: float64

In [12]:
residuals = pd.DataFrame([cross['id.1'], mdf.resid]).T

In [15]:
residuals.columns = ['id', 'sex_gen_resid']
residuals.to_csv('20201125_residuals_precorrected_for_sex_and_generation.csv')